# eSafety Hackathon - Classification Approaches

This notebook demonstrates different approaches to classifying personas.

## Approaches Covered
1. **Perspective API** - Quick toxicity baseline
2. **LLM Classification** - Flexible, prompt-based
3. **Embedding + Clustering** - Unsupervised grouping
4. **Keyword/Pattern Matching** - Fast, explainable baseline

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')

from dotenv import load_dotenv
load_dotenv('../.env')

# Sample data for testing
test_messages = [
    "I just got promoted to VP after only 6 months! Hard work pays off. Thoughts? 🚀💪",
    "I hate my body. Why can't I look like the models I see online?",
    "Women only want one thing and it's not personality. The game is rigged.",
    "Had a nice walk in the park today. The weather was perfect! 🌳",
    "I'm HUMBLED to announce that I've been named a Top 30 Under 30 Thought Leader in Digital Transformation. None of this would be possible without my amazing team. Agree that teamwork makes the dream work? 🙏✨",
    "Skipped dinner again. Need to lose these last 5 pounds before the wedding.",
    "Society is designed to keep us down. Wake up.",
    "Just finished a great book! Would recommend to anyone interested in history."
]

print(f"Testing with {len(test_messages)} sample messages")

## Approach 1: Keyword/Pattern Matching (Fast Baseline)

Simple but explainable. Good starting point.

In [ ]:
import re

def classify_by_keywords(text: str) -> dict:
    """
    Simple keyword-based classification.
    Returns dict with scores for each persona.
    """
    text_lower = text.lower()
    
    scores = {
        'linkedin_lunatic': 0,
        'body_dysmorphia': 0,
        'incel': 0,
        'normal': 0
    }
    
    # LinkedIn Lunatic indicators
    linkedin_patterns = [
        r'agree\s*\?',
        r'thoughts\s*\?',
        r"i('m| am) (humbled|honored|blessed|grateful)",
        r'thought leader',
        r'game.?changer',
        r'dream job',
        r'promoted',
        r'🚀|💪|🙏|✨',
        r'hard work pays off',
        r'none of this would be possible'
    ]
    for pattern in linkedin_patterns:
        if re.search(pattern, text_lower) or re.search(pattern, text):
            scores['linkedin_lunatic'] += 1
    
    # Body dysmorphia indicators
    body_patterns = [
        r'hate my body',
        r'(wish i|if only i) (looked|was|could be)',
        r'(skipped|skip) (lunch|dinner|breakfast|meals?)',
        r'need to lose',
        r'too (fat|skinny|ugly)',
        r"can't look like",
        r'(thinner|skinnier|prettier|more attractive)',
        r'fasting',
        r'calories'
    ]
    for pattern in body_patterns:
        if re.search(pattern, text_lower):
            scores['body_dysmorphia'] += 1
    
    # Incel indicators (be careful with these - context matters)
    incel_patterns = [
        r'(women|females?) only want',
        r'rigged',
        r'society (is|designed to)',
        r'wake up',
        r'blackpill',
        r'chad',
        r'normie',
        r'keep us down'
    ]
    for pattern in incel_patterns:
        if re.search(pattern, text_lower):
            scores['incel'] += 1
    
    # Determine primary category
    max_score = max(scores.values())
    if max_score == 0:
        scores['normal'] = 1
        primary = 'normal'
    else:
        primary = max(scores, key=scores.get)
    
    return {
        'primary_category': primary,
        'scores': scores,
        'confidence': max_score / 5  # Normalize to 0-1 range
    }

# Test it
print("Keyword-based Classification Results:\n")
for msg in test_messages:
    result = classify_by_keywords(msg)
    print(f"Category: {result['primary_category']} (conf: {result['confidence']:.2f})")
    print(f"  Text: {msg[:80]}...\n")

## Approach 2: Perspective API (Toxicity Baseline)

**Note**: Requires API key. Get one at https://developers.perspectiveapi.com/

In [ ]:
# Uncomment and run if you have a Perspective API key

# from src.classifiers.api_classifiers import PerspectiveAPI

# try:
#     perspective = PerspectiveAPI()
#     
#     print("Perspective API Analysis:\n")
#     for msg in test_messages[:3]:  # Limit to avoid rate limits
#         scores = perspective.analyze(msg, attributes=['TOXICITY', 'INSULT', 'IDENTITY_ATTACK'])
#         print(f"Text: {msg[:60]}...")
#         for attr, score in scores.items():
#             print(f"  {attr}: {score:.3f}")
#         print()
# except Exception as e:
#     print(f"Perspective API not configured: {e}")

## Approach 3: LLM Classification (Flexible & Powerful)

Use GPT-4 or Claude for nuanced classification.

In [ ]:
# Uncomment and run if you have an OpenAI or Anthropic API key

# from src.classifiers.api_classifiers import LLMClassifier

# try:
#     # Use OpenAI (gpt-4o-mini is fast and cheap)
#     classifier = LLMClassifier(provider='openai', model='gpt-4o-mini')
#     
#     # Or use Anthropic
#     # classifier = LLMClassifier(provider='anthropic')
#     
#     print("LLM Classification Results:\n")
#     for msg in test_messages:
#         result = classifier.classify_persona(msg)
#         print(f"Category: {result.get('category')} (conf: {result.get('confidence', 'N/A')})")
#         print(f"  Indicators: {result.get('indicators', [])}")
#         print(f"  Text: {msg[:60]}...\n")
# except Exception as e:
#     print(f"LLM not configured: {e}")

## Approach 4: Embedding + Clustering

Group similar messages together without predefined categories.

In [ ]:
# This uses sentence-transformers (runs locally, no API needed)

try:
    from sentence_transformers import SentenceTransformer
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    import matplotlib.pyplot as plt
    
    # Load model (first time will download ~90MB)
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Get embeddings
    embeddings = model.encode(test_messages)
    print(f"Embedding shape: {embeddings.shape}")
    
    # Cluster
    n_clusters = 3
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings)
    
    # Visualize with PCA
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap='viridis', s=100)
    
    # Add labels
    for i, txt in enumerate(test_messages):
        plt.annotate(f"{i}: {txt[:30]}...", (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=8)
    
    plt.colorbar(scatter, label='Cluster')
    plt.title('Message Embeddings (PCA)')
    plt.show()
    
    # Show clusters
    print("\nCluster assignments:")
    for cluster in range(n_clusters):
        print(f"\nCluster {cluster}:")
        for i, (msg, label) in enumerate(zip(test_messages, labels)):
            if label == cluster:
                print(f"  - {msg[:70]}...")

except ImportError:
    print("sentence-transformers not installed. Run: pip install sentence-transformers")

## Combining Approaches

Best results often come from combining multiple signals.

In [ ]:
def classify_ensemble(text: str) -> dict:
    """
    Combine multiple classification signals.
    """
    results = {}
    
    # 1. Keyword baseline
    keyword_result = classify_by_keywords(text)
    results['keyword'] = keyword_result['primary_category']
    results['keyword_confidence'] = keyword_result['confidence']
    
    # 2. Add Perspective API scores if available
    # results['toxicity'] = perspective_score
    
    # 3. Add LLM classification if available
    # results['llm'] = llm_result
    
    # 4. Feature-based signals
    from src.utils.text_utils import count_features, detect_patterns
    features = count_features(text)
    patterns = detect_patterns(text)
    
    results['emoji_heavy'] = features.get('emoji_count', 0) > 3
    results['shouting'] = patterns.get('shouting', False)
    results['engagement_bait'] = patterns.get('engagement_bait', False)
    
    return results

# Test ensemble
print("Ensemble Classification:\n")
for msg in test_messages:
    result = classify_ensemble(msg)
    print(f"Text: {msg[:60]}...")
    for key, value in result.items():
        print(f"  {key}: {value}")
    print()

## Scaling Up

Tips for processing millions of messages:

1. **Start with keyword baseline** - Fast, runs on everything
2. **Use embeddings for clustering** - Can process locally, no API limits
3. **Reserve LLM calls for uncertain cases** - Most expensive, use strategically
4. **Batch processing** - Use async/parallel where possible
5. **Sample first** - Test on 1000 messages before running on millions

In [ ]:
# Example: Processing a large dataset efficiently

def process_large_dataset(df: pd.DataFrame, text_column: str = 'text') -> pd.DataFrame:
    """
    Process a large dataset with tiered classification.
    """
    from tqdm import tqdm
    tqdm.pandas()
    
    # Step 1: Fast keyword classification for everything
    print("Step 1: Keyword classification...")
    df['keyword_result'] = df[text_column].progress_apply(classify_by_keywords)
    df['primary_category'] = df['keyword_result'].apply(lambda x: x['primary_category'])
    df['confidence'] = df['keyword_result'].apply(lambda x: x['confidence'])
    
    # Step 2: Flag uncertain cases for LLM review
    uncertain_mask = df['confidence'] < 0.4
    print(f"\nStep 2: {uncertain_mask.sum()} uncertain cases flagged for LLM review")
    
    # Step 3: LLM classification on uncertain cases (if API available)
    # df.loc[uncertain_mask, 'llm_result'] = ...
    
    return df

# Test on sample
sample_df = pd.DataFrame({'text': test_messages})
result_df = process_large_dataset(sample_df)
result_df[['text', 'primary_category', 'confidence']]

## Next Steps

1. **Get the real data** from the hackathon platform
2. **Explore unique vocabulary** - What words/phrases appear only in certain personas?
3. **Look at user-level patterns** - Aggregate messages per user for more signal
4. **Build a simple UI** - For the demo, consider Streamlit or Gradio